In [1]:
import pandas as pd
import cv2
import urllib.request
import numpy as np
import os
from datetime import datetime
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Đường dẫn ảnh và URL camera
path = r"E:\ATTENDANCE\image_folder"
url = 'http://192.168.1.17/cam-mid.jpg'


In [3]:
# Load ảnh và tên lớp
images = []
classNames = []
myList = os.listdir(path)
print("Loaded image list:", myList)

for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print("Class names:", classNames)

Loaded image list: ['ellon musk.jfif', 'priyansh.jfif', 'usain bolt.jpg']
Class names: ['ellon musk', 'priyansh', 'usain bolt']


In [7]:
# Kiểm tra và khởi tạo file Attendance.csv
if 'Attendance.csv' in os.listdir(os.path.join(os.getcwd(), 'attendace')):
    print("Attendance.csv exists. Removing old file...")
    os.remove("Attendance.csv")
else:
    df = pd.DataFrame(columns=["Name", "Time"])
    df.to_csv("Attendance.csv", index=False)

In [4]:
# Dùng MTCNN để detect face và InceptionResnetV1 để tạo embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(image_size=160, margin=20, min_face_size=20, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Tạo encoding cho các ảnh đã load
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_cropped = mtcnn(img)
        if img_cropped is not None:
            with torch.no_grad():
                encoding = resnet(img_cropped.unsqueeze(0).to(device))
                encodeList.append(encoding.squeeze(0).cpu().numpy())
    return encodeList



100%|██████████| 107M/107M [00:58<00:00, 1.90MB/s] 


In [5]:
# Tìm encoding cho ảnh đầu vào
encodeListKnown = findEncodings(images)
print('Encoding Complete')

Encoding Complete
